## google email 


jeans site:amazon.com "@gmail.com"


普遍邮箱适用的正则表达式

reg = r'[A-Za-z0-9_.]+@[A-Za-z0-9]+\.[a-z]{3}'

因为

\w 等同于[a-z0-9A-Z_]匹配大小写字母、数字和下划线

所以

reg = r'\w+@\w+.[a-z]{3}

但这个没有带点的

In [ ]:
import time,random,re,csv
from pyquery import PyQuery as pq


def email(url):

    doc = pq(url)
    #print(doc)
    content = doc('.st')
    print(content)
    

url = 'https://www.google.com.hk/search?safe=strict&biw=1920&bih=889&ei=d3NKXMXkF6LB7gLp6ZX4Dw&q=jeans+site%3Aamazon.com+%22%40gmail.com%22&oq=jeans+site%3Aamazon.com+%22%40gmail.com%22&gs_l=psy-ab.3...9441.11399..11908...0.0..0.237.2275.2-10......0....1..gws-wiz.AJwyto52dO4'
email(url)

### 省略

为了向您显示相关程度最高的结果，我们省略了一些与已显示的 14 条结果极为相似的条目。
如有需要，您可以重新搜索以显示省略的结果。

In [18]:
from selenium import webdriver
import time,random,re,csv,pymongo
from pyquery import PyQuery as pq
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

    
def get_email(driver,url,save_file,page_num):
    
    driver.get(url) # 访问网页
    print('open crawl page... {}'.format(url))
    time.sleep(2)    
    jsCode = "var q=document.documentElement.scrollTop=100000"
    driver.execute_script(jsCode)
    html = driver.page_source
    doc = pq(html)
    current_page = int(driver.find_element_by_css_selector('td.cur').text)
    print('current page is {}'.format(current_page))
    
    content = doc.find('.st')
    with open(save_file,'a',encoding='utf-8') as w:
        for i in content.items():
            text = re.sub(r'\n','',pq(i.html()).text()) 
            try:
                email = re.search(r'\w+@\w+.[a-z]{3}',text).group() 
                print('email found {}'.format(email))
                search = {
                    'email': email
                }
                save_to_mongo(search)
                #w.write('{}\n'.format(email))
            except AttributeError:
                print('no email found')
    
    try:
        next_page = driver.find_element_by_id('pnnext').get_attribute('href')
        if current_page < page_num :
            print('next page url: {}'.format(next_page))
            time.sleep(random.randint(2,4))
            get_email(driver,next_page,save_file,page_num)     
    except NoSuchElementException:
        print('there is no more pages.')
        
    
    try:
        next_page = driver.find_element_by_id('pnnext').get_attribute('href')
        if current_page < page_num :
            print('next page url: {}'.format(next_page))
            time.sleep(random.randint(2,4))
            get_email(driver,next_page,save_file,page_num)     
    except NoSuchElementException:
        print('there is no more pages.')    
    
            
def save_to_mongo(result):
    MONGO_URL = 'localhost'
    MONGO_DB = 'shopcrawl'
    MONGO_COLLECTION = 'email'
    client = pymongo.MongoClient(MONGO_URL)
    db = client[MONGO_DB]
    collection = db[MONGO_COLLECTION]
    try:
        if collection.insert_one(result):
            print('save to mongodb')
    except Exception:
        print('failed to save to mongodb')


#url = 'https://www.google.com.hk/search?safe=strict&biw=1920&bih=889&ei=d3NKXMXkF6LB7gLp6ZX4Dw&q=jeans+site%3Aamazon.com+%22%40gmail.com%22&oq=jeans+site%3Aamazon.com+%22%40gmail.com%22&gs_l=psy-ab.3...9441.11399..11908...0.0..0.237.2275.2-10......0....1..gws-wiz.AJwyto52dO4'
url_1 = 'https://www.google.com.hk/search?safe=strict&ei=KYpKXKzPCPXY9AOd44GwBw&q=jeans+site%3Awish.com+%22%40gmail.com%22&oq=jeans+site%3Awish.com+%22%40gmail.com%22&gs_l=psy-ab.3...8574389.8575320..8575770...0.0..0.225.1094.2-5......0....1..gws-wiz.TUS1hiC8cxo'
save_file = r'email_result.csv'
page_num = 50

def goole_email(url,save_file,page_num):
    
    #chrome_options = Options()
    #chrome_options.add_argument('--headless')
    #driver = webdriver.Chrome(chrome_options=chrome_options) 
    driver = webdriver.Chrome() 
    get_email(driver,url,save_file,page_num)
    driver.close()
    print('getting email done')
    
goole_email(url_1,save_file,page_num)

open crawl page... https://www.google.com.hk/search?safe=strict&ei=KYpKXKzPCPXY9AOd44GwBw&q=jeans+site%3Awish.com+%22%40gmail.com%22&oq=jeans+site%3Awish.com+%22%40gmail.com%22&gs_l=psy-ab.3...8574389.8575320..8575770...0.0..0.225.1094.2-5......0....1..gws-wiz.TUS1hiC8cxo
current page is 1
email found budbittner16@gmail.com
save to mongodb
no email found
no email found
email found miaantunez1987@gmail.com
save to mongodb
email found miaantunez1987@gmail.com
save to mongodb
email found justaregularemail@gmail.com
save to mongodb
email found hrosser90@gmail.com
save to mongodb
no email found
no email found
next page url: https://www.google.com.hk/search?q=jeans+site:wish.com+%22%40gmail.com%22&safe=strict&ei=QcdKXNSVL5O50PEPmsODYA&start=10&sa=N&ved=0ahUKEwiUuq23wIjgAhWTHDQIHZrhAAwQ8NMDCH8
open crawl page... https://www.google.com.hk/search?q=jeans+site:wish.com+%22%40gmail.com%22&safe=strict&ei=QcdKXNSVL5O50PEPmsODYA&start=10&sa=N&ved=0ahUKEwiUuq23wIjgAhWTHDQIHZrhAAwQ8NMDCH8
current page

In [ ]:
from selenium import webdriver
import time,random,re,csv,pymongo
from pyquery import PyQuery as pq
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

    
def get_email(driver,url,save_file,page_num):
    
    driver.get(url) # 访问网页
    print('open crawl page... {}'.format(url))
    time.sleep(2)    
    jsCode = "var q=document.documentElement.scrollTop=100000"
    driver.execute_script(jsCode)
    html = driver.page_source
    doc = pq(html)
    current_page = int(driver.find_element_by_css_selector('td.cur').text)
    print('current page is {}'.format(current_page))
    
    content = doc.find('.st')
    with open(save_file,'a',encoding='utf-8') as w:
        for i in content.items():
            text = re.sub(r'\n','',pq(i.html()).text()) 
            try:
                email = re.search(r'\w+@\w+.[a-z]{3}',text).group() 
                print('email found {}'.format(email))
                search = {
                    'email': email
                }
                save_to_mongo(search)
                #w.write('{}\n'.format(email))
            except AttributeError:
                print('no email found')
    
    try:
        next_page = driver.find_element_by_id('ofr i a').get_attribute('href')
        if current_page < page_num :
            print('next page url: {}'.format(next_page))
            time.sleep(random.randint(2,4))
            get_email(driver,next_page,save_file,page_num)     
    except NoSuchElementException:
        print('there is no more pages.')
        
    
    
    try:
        next_page = driver.find_element_by_id('pnnext').get_attribute('href')
        if current_page < page_num :
            print('next page url: {}'.format(next_page))
            time.sleep(random.randint(2,4))
            get_email(driver,next_page,save_file,page_num)     
    except NoSuchElementException:
        print('there is no more pages.')    
    
            
def save_to_mongo(result):
    MONGO_URL = 'localhost'
    MONGO_DB = 'shopcrawl'
    MONGO_COLLECTION = 'email'
    client = pymongo.MongoClient(MONGO_URL)
    db = client[MONGO_DB]
    collection = db[MONGO_COLLECTION]
    try:
        if collection.insert_one(result):
            print('save to mongodb')
    except Exception:
        print('failed to save to mongodb')


#url = 'https://www.google.com.hk/search?safe=strict&biw=1920&bih=889&ei=d3NKXMXkF6LB7gLp6ZX4Dw&q=jeans+site%3Aamazon.com+%22%40gmail.com%22&oq=jeans+site%3Aamazon.com+%22%40gmail.com%22&gs_l=psy-ab.3...9441.11399..11908...0.0..0.237.2275.2-10......0....1..gws-wiz.AJwyto52dO4'
url_1 = 'https://www.google.com.hk/search?safe=strict&ei=KYpKXKzPCPXY9AOd44GwBw&q=jeans+site%3Awish.com+%22%40gmail.com%22&oq=jeans+site%3Awish.com+%22%40gmail.com%22&gs_l=psy-ab.3...8574389.8575320..8575770...0.0..0.225.1094.2-5......0....1..gws-wiz.TUS1hiC8cxo'
save_file = r'email_result.csv'
page_num = 50

def goole_email(url,save_file,page_num):
    
    #chrome_options = Options()
    #chrome_options.add_argument('--headless')
    #driver = webdriver.Chrome(chrome_options=chrome_options) 
    driver = webdriver.Chrome() 
    get_email(driver,url,save_file,page_num)
    driver.close()
    print('getting email done')
    
goole_email(url_1,save_file,page_num)

## gmail 爬取脚本 可行

In [19]:
from selenium import webdriver
import time,random,re,csv,pymongo
from pyquery import PyQuery as pq
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

    
def get_email(driver,url,save_file,page_num):
    
    driver.get(url) # 访问网页
    print('open crawl page... {}'.format(url))
    time.sleep(2)    
    jsCode = "var q=document.documentElement.scrollTop=100000"
    driver.execute_script(jsCode)
    html = driver.page_source
    doc = pq(html)
    current_page = int(driver.find_element_by_css_selector('td.cur').text)
    print('current page is {}'.format(current_page))
    
    content = doc.find('.st')
    with open(save_file,'a',encoding='utf-8') as w:
        for i in content.items():
            text = re.sub(r'\n','',pq(i.html()).text()) 
            try:
                email = re.search(r'\w+@\w+.[a-z]{3}',text).group() 
                print('email found {}'.format(email))
                search = {
                    'email': email
                }
                save_to_mongo(search)
                #w.write('{}\n'.format(email))
            except AttributeError:
                print('no email found')
    
    try:
        next_page = driver.find_element_by_id('pnnext').get_attribute('href')
        if current_page < page_num :
            print('next page url: {}'.format(next_page))
            time.sleep(random.randint(2,4))
            get_email(driver,next_page,save_file,page_num)     
    except NoSuchElementException:
        print('there is no more pages.')

            
def save_to_mongo(result):
    MONGO_URL = 'localhost'
    MONGO_DB = 'shopcrawl'
    MONGO_COLLECTION = 'email'
    client = pymongo.MongoClient(MONGO_URL)
    db = client[MONGO_DB]
    collection = db[MONGO_COLLECTION]
    try:
        if collection.insert_one(result):
            print('save to mongodb')
    except Exception:
        print('failed to save to mongodb')


url = 'https://www.google.com.hk/search?safe=strict&biw=1920&bih=889&ei=d3NKXMXkF6LB7gLp6ZX4Dw&q=jeans+site%3Aamazon.com+%22%40gmail.com%22&oq=jeans+site%3Aamazon.com+%22%40gmail.com%22&gs_l=psy-ab.3...9441.11399..11908...0.0..0.237.2275.2-10......0....1..gws-wiz.AJwyto52dO4'
save_file = r'email_result.csv'
page_num = 50

def goole_email(url,save_file,page_num):
    
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    driver = webdriver.Chrome(chrome_options=chrome_options) 
    #driver = webdriver.Chrome() 
    get_email(driver,url,save_file,page_num)
    driver.close()
    print('getting email done')
    
goole_email(url,save_file,page_num)

open crawl page... https://www.google.com.hk/search?safe=strict&biw=1920&bih=889&ei=d3NKXMXkF6LB7gLp6ZX4Dw&q=jeans+site%3Aamazon.com+%22%40gmail.com%22&oq=jeans+site%3Aamazon.com+%22%40gmail.com%22&gs_l=psy-ab.3...9441.11399..11908...0.0..0.237.2275.2-10......0....1..gws-wiz.AJwyto52dO4
current page is 1
email found lbts@gmail.com
save to mongodb
email found magekagura9@gmail.com
save to mongodb
email found saadbenchkr@gmail.com
save to mongodb
email found dakilivewallpapers@gmail.com
save to mongodb
email found breakthewineglass@gmail.com
save to mongodb
email found mikimarcu256@gmail.com
save to mongodb
email found Jsokolis@gmail.com
save to mongodb
email found breakthewineglass@gmail.com
save to mongodb
email found evertvear@gmail.com
save to mongodb
next page url: https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=18pKXLLdBsXo9AOb7rmoBQ&start=10&sa=N&ved=0ahUKEwjypOPsw4jgAhVFNH0KHRt3DlUQ8NMDCHc
open crawl page... https://www.

current page is 10
email found msgajraj1984@gmail.com
save to mongodb
email found eddieriva@gmail.com
save to mongodb
email found msf0wl3@gmail.com
save to mongodb
email found visanuw86@gmail.com
save to mongodb
email found msgajraj1984@gmail.com
save to mongodb
email found madhukar262@gmail.com
save to mongodb
email found ahailu10@gmail.com
save to mongodb
email found aruboy56@gmail.com
save to mongodb
email found delisafashion2020@gmail.com
save to mongodb
email found Fashionscrubs4u@gmail.com
save to mongodb
next page url: https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=FstKXMjjHPjC0PEP--uSsAc&start=100&sa=N&ved=0ahUKEwjIxv6KxIjgAhV4ITQIHfu1BHY4WhDw0wMIaA
open crawl page... https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=FstKXMjjHPjC0PEP--uSsAc&start=100&sa=N&ved=0ahUKEwjIxv6KxIjgAhV4ITQIHfu1BHY4WhDw0wMIaA
current page is 11
email found amorebeaute2012@gmail.com
sa

open crawl page... https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=UMtKXObwCoCR0PEPnNe36AQ&start=190&sa=N&ved=0ahUKEwjm2MCmxIjgAhWACDQIHZzrDU04tAEQ8NMDCGQ
current page is 20
email found user@gmail.com
save to mongodb
email found thatsgrandluxuries@gmail.com
save to mongodb
email found ReviewOne23@gmail.com
save to mongodb
email found vinodsb25@gmail.com
save to mongodb
no email found
email found harlesshouse3@gmail.com
save to mongodb
email found cohenmayer7@gmail.com
save to mongodb
email found bengoosales@gmail.com
save to mongodb
email found delisafashion2020@gmail.com
save to mongodb
email found barryzrose@gmail.com
save to mongodb
next page url: https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=V8tKXIq9EK3M0PEP3Iu2wAE&start=200&sa=N&ved=0ahUKEwjKxPGpxIjgAhUtJjQIHdyFDRg4vgEQ8NMDCG0
open crawl page... https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gma

## 显示省略页面 可行
为了向您显示相关程度最高的结果，我们省略了一些与已显示的 14 条结果极为相似的条目。
如有需要，您可以重新搜索以显示省略的结果。

In [1]:
from selenium import webdriver
import time,random,re,csv,pymongo
from pyquery import PyQuery as pq
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

    
def get_email(driver,url,save_file,page_num):
    
    driver.get(url) # 访问网页
    print('open crawl page... {}'.format(url))
    time.sleep(2)    
    jsCode = "var q=document.documentElement.scrollTop=100000"
    driver.execute_script(jsCode)
    time.sleep(2)
    html = driver.page_source
    #print(html)
    doc = pq(html)
    current_page = int(driver.find_element_by_css_selector('td.cur').text)
    print('current page is {}'.format(current_page))
    
    content = doc.find('.st')
    with open(save_file,'a',encoding='utf-8') as w:
        for i in content.items():
            text = re.sub(r'\n','',pq(i.html()).text()) 
            try:
                email = re.search(r'\w+@\w+.[a-z]{3}',text).group() 
                print('email found {}'.format(email))
                data = {
                    'email': email
                }
                collection.update({'email':email},{'$set':data},True)
                #w.write('{}\n'.format(email))
            except AttributeError:
                print('no email found')
    
    try:
        next_page = driver.find_element_by_id('pnnext').get_attribute('href')
        if current_page < page_num :
            print('next page url: {}'.format(next_page))
            time.sleep(random.randint(20,60))
            get_email(driver,next_page,save_file,page_num)     
    except NoSuchElementException:
        driver.save_screenshot('exception.png')
        try:
            html = driver.page_source
            print(html)
            driver.find_element_by_id('recaptcha').click()
            print('google verification I am not a robot')
            html = driver.page_source
            print(html)
            get_email(driver,next_page,save_file,page_num)
        except:
            print('there is no more pages.')
        
        
    try:
        omit_url = driver.find_element_by_css_selector('#ofr a').get_attribute('href')
        print('重新搜索以显示省略的结果: {}'.format(omit_url))
        get_email(driver,omit_url,save_file,page_num)
    except:
        pass


def google_email(url,save_file,page_num):
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    driver = webdriver.Chrome(chrome_options=chrome_options) 
    get_email(driver,url,save_file,page_num)
    driver.close()
    print('getting email done')

    
MONGO_URL = 'localhost'
MONGO_DB = 'shopcrawl'
MONGO_COLLECTION = 'gmail_amazon_jeans'
client = pymongo.MongoClient(MONGO_URL)
db = client[MONGO_DB]
collection = db[MONGO_COLLECTION] 

url = 'https://www.google.com.hk/search?safe=strict&biw=1920&bih=889&ei=d3NKXMXkF6LB7gLp6ZX4Dw&q=jeans+site%3Aamazon.com+%22%40gmail.com%22&oq=jeans+site%3Aamazon.com+%22%40gmail.com%22&gs_l=psy-ab.3...9441.11399..11908...0.0..0.237.2275.2-10......0....1..gws-wiz.AJwyto52dO4'
save_file = r'amazon_jeans_gmail_result_norepeat.csv'
page_num = 50
google_email(url,save_file,page_num)

open crawl page... https://www.google.com.hk/search?safe=strict&biw=1920&bih=889&ei=d3NKXMXkF6LB7gLp6ZX4Dw&q=jeans+site%3Aamazon.com+%22%40gmail.com%22&oq=jeans+site%3Aamazon.com+%22%40gmail.com%22&gs_l=psy-ab.3...9441.11399..11908...0.0..0.237.2275.2-10......0....1..gws-wiz.AJwyto52dO4
current page is 1
email found lbts@gmail.com
email found magekagura9@gmail.com
email found dakilivewallpapers@gmail.com
email found breakthewineglass@gmail.com
email found Jsokolis@gmail.com
email found mikimarcu256@gmail.com
email found toqeer65@gmail.com
email found breakthewineglass@gmail.com
no email found
next page url: https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=dH1iXIDFArLF0PEPir-Z0AM&start=10&sa=N&ved=0ahUKEwiAz4-23bXgAhWyIjQIHYpfBjoQ8NMDCHg


c:\users\win7\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


open crawl page... https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=dH1iXIDFArLF0PEPir-Z0AM&start=10&sa=N&ved=0ahUKEwiAz4-23bXgAhWyIjQIHYpfBjoQ8NMDCHg
current page is 2
email found sarifhidayat808@gmail.com
email found evertvear@gmail.com
email found dakilivewallpapers@gmail.com
email found tnvo18@gmail.com
no email found
email found tkrysl@gmail.com
email found chamsimo17@gmail.com
no email found
email found vasishtanagalla@gmail.com
no email found
next page url: https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=kH1iXL7eCbWy0PEP9caL6AE&start=20&sa=N&ved=0ahUKEwi-5sPD3bXgAhU1GTQIHXXjAh04ChDw0wMIbQ
open crawl page... https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=kH1iXL7eCbWy0PEP9caL6AE&start=20&sa=N&ved=0ahUKEwi-5sPD3bXgAhU1GTQIHXXjAh04ChDw0wMIbQ
current page is 3
email found vyperapps@gmail.com
email found tkr

open crawl page... https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=L39iXJm5MKyy0PEPr52o6Ag&start=120&sa=N&ved=0ahUKEwjZjNyJ37XgAhUsGTQIHa8OCo04bhDw0wMIdQ
current page is 13
no email found
email found Fashionscrubs4u@gmail.com
email found Fashionscrubs4u@gmail.com
email found jjjbrown777@gmail.com
no email found
email found mmdoridior@gmail.com
no email found
email found Fashionscrubs4u@gmail.com
email found stellabagwell@gmail.com
no email found
next page url: https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=TX9iXKfBOMXA0PEPu5GNOA&start=130&sa=N&ved=0ahUKEwjnm4uY37XgAhVFIDQIHbtIAwc4eBDw0wMIcA
open crawl page... https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=TX9iXKfBOMXA0PEPu5GNOA&start=130&sa=N&ved=0ahUKEwjnm4uY37XgAhVFIDQIHbtIAwc4eBDw0wMIcA
current page is 14
email found Krisstodd13@gmail.com
no email found


open crawl page... https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&filter=0
current page is 1
email found lbts@gmail.com
email found magekagura9@gmail.com
no email found
no email found
email found breakthewineglass@gmail.com
email found dakilivewallpapers@gmail.com
email found mikimarcu256@gmail.com
no email found
email found breakthewineglass@gmail.com
next page url: https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=-IBiXPG7OozA0PEPpruC6Ak&start=10&sa=N&filter=0&ved=0ahUKEwjxl9vj4LXgAhUMIDQIHaadAJ0Q8NMDCHs
open crawl page... https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=-IBiXPG7OozA0PEPpruC6Ak&start=10&sa=N&filter=0&ved=0ahUKEwjxl9vj4LXgAhUMIDQIHaadAJ0Q8NMDCHs
current page is 2
email found Jsokolis@gmail.com
email found tnvo18@gmail.com
email found toqeer65@gmail.com
email found evertvear@gmail.com
email found 

open crawl page... https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=uYJiXM5lzb3Q8Q-XlanoDw&start=110&sa=N&filter=0&ved=0ahUKEwiOpq654rXgAhXNHjQIHZdKCv04ZBDw0wMIbw
current page is 12
email found amorebeaute2012@gmail.com
email found jamandspoon@gmail.com
email found yksliuming@gmail.com
email found Lunamoonbridal@gmail.com
email found billgorden2@gmail.com
email found bigjackgrizzly@gmail.com
email found prguyvic@gmail.com
email found Shiloh@gmail.com
email found Shiloh@gmail.com
no email found
next page url: https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=8YJiXM6BELqz0PEPibOekAw&start=120&sa=N&filter=0&ved=0ahUKEwiOvpfU4rXgAhW6GTQIHYmZB8I4bhDw0wMIbg
open crawl page... https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=8YJiXM6BELqz0PEPibOekAw&start=120&sa=N&filter=0&ved=0ahUKEwiOvpfU4rXgAhW6GTQIHYmZB8I4bhDw0wMIbg


## 省略会出现验证

In [42]:
from selenium import webdriver
import time,random,re,csv,pymongo
from pyquery import PyQuery as pq
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

    
def get_email(driver,url,save_file,page_num):
    
    try:
        driver.get(url) # 访问网页
        print('open crawl page... {}'.format(url))
        time.sleep(5)    
        jsCode = "var q=document.documentElement.scrollTop=100000"
        driver.execute_script(jsCode)
        time.sleep(2)
        html = driver.page_source
        #print(html)
        doc = pq(html)
        current_page = int(driver.find_element_by_css_selector('td.cur').text)
        print('current page is {}'.format(current_page))
        content = doc.find('.st')
        with open(save_file,'a',encoding='utf-8') as w:
            for i in content.items():
                #text = re.sub(r'\n','',pq(i.html()).text()) 
                text = re.sub(r'\n','',i.text()) 
                try:
                    email = re.search(r'[A-Za-z0-9_.]+@[A-Za-z0-9]+\.[a-z]{3}',text).group() 
                    print('email found {}'.format(email))
                    data = {
                        'category':'jeans',
                        'email': email
                    }
                    collection.update_one({'email':email},{'$set':data},True)
                    print('save to mongodb')
                    w.write('{},{}\n'.format('jeans',email))
                except AttributeError:
                    print('no email found')

        try:
            next_page = driver.find_element_by_id('pnnext').get_attribute('href')
            if current_page < page_num :
                print('next page url: {}'.format(next_page))
                time.sleep(random.randint(30,60))
                get_email(driver,next_page,save_file,page_num)     
        except NoSuchElementException:
            driver.save_screenshot('exception.png')
            print('there is no more pages.')
    except NoSuchElementException as err:
        print('程序出问题，捕获截图...',err)
        driver.save_screenshot('exception_start.png')
        
def google_email(url,save_file,page_num):
    #chrome_options = Options()
    #chrome_options.add_argument('--headless')
    #chrome_options.add_argument('user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"')
    #driver = webdriver.Chrome(chrome_options=chrome_options) 
    driver = webdriver.Chrome() 
    get_email(driver,url,save_file,page_num)
    driver.close()
    print('getting email done')

    
MONGO_URL = 'localhost'
MONGO_DB = 'emailcrawl'
MONGO_COLLECTION = 'gmail_jeans'
client = pymongo.MongoClient(MONGO_URL)
db = client[MONGO_DB]
collection = db[MONGO_COLLECTION] 


save_file = r'gmail_jeans.csv'
page_num = 50
#url = 'https://www.google.com.hk/search?q=jeans+site:facebook.com+%22%40gmail.com%22&ei=yspjXNy6AqK70PEP6YqDmAI&start=80&sa=N&ved=0ahUKEwjcx8Wom7jgAhWiHTQIHWnFACM4RhDw0wMIWw'
#google_email(url,save_file,page_num)


keyword_list = ['jeans site:amazon.com "@gmail.com"','jeans site:wish.com "@gmail.com"','jeans site:ebay.com "@gmail.com"','jeans site:facebook.com "@gmail.com"','jeans site:Instagram.com "@gmail.com"','jeans site:twitter.com "@gmail.com"','jeans site:tumblr.com "@gmail.com"','jeans site:linkedin.com "@gmail.com"','Jeans site:amazon.com @hotmail.com','Jeans site:wish.com @hotmail.com','Jeans site:ebay.com @hotmail.com','Jeans site:facebook.com @hotmail.com','Jeans site:Instagram.com @hotmail.com','Jeans site:twitter.com @hotmail.com','Jeans site:tumblr.com @hotmail.com','Jeans site:linkedin.com @hotmail.com','Jeans site:amazon.com @yahoo.com','Jeans site:wish.com @yahoo.com','Jeans site:ebay.com @yahoo.com','Jeans site:facebook.com @yahoo.com','Jeans site:Instagram.com @yahoo.com','Jeans site:twitter.com @yahoo.com','Jeans site:tumblr.com @yahoo.com','Jeans site:linkedin.com @yahoo.com']
url_format = 'https://www.google.com.hk/search?&q={0[0]}+site%3A{0[1]}+%22%40{0[2]}%22&oq'
for i in keyword_list[-5:]:
    keyword = re.sub(r'site:|@|\"','',i,count=0)
    keywords = keyword.split()
    url = url_format.format(keywords)
    google_email(url,save_file,page_num)

open crawl page... https://www.google.com.hk/search?&q=Jeans+site%3Afacebook.com+%22%40yahoo.com%22&oq
current page is 1
no email found
email found awco.woven@yahoo.com
save to mongodb
no email found
email found awco.woven@yahoo.com
save to mongodb
email found elengie.jeans@yahoo.com
save to mongodb
email found awco.woven@yahoo.com
save to mongodb
no email found
email found blendaseitz@yahoo.com
save to mongodb
email found www.whalejeans@yahoo.com.tw
save to mongodb
email found elengie.jeans@yahoo.com
save to mongodb
next page url: https://www.google.com.hk/search?q=Jeans+site:facebook.com+%22%40yahoo.com%22&safe=strict&ei=osxkXKCOA5br-Qa455rIDQ&start=10&sa=N&ved=0ahUKEwig586bkbrgAhWWdd4KHbizBtkQ8NMDCF8
open crawl page... https://www.google.com.hk/search?q=Jeans+site:facebook.com+%22%40yahoo.com%22&safe=strict&ei=osxkXKCOA5br-Qa455rIDQ&start=10&sa=N&ved=0ahUKEwig586bkbrgAhWWdd4KHbizBtkQ8NMDCF8
current page is 2
email found elengie.jeans@yahoo.com
save to mongodb
email found ctackett7@y

open crawl page... https://www.google.com.hk/search?q=Jeans+site:facebook.com+%22%40yahoo.com%22&safe=strict&ei=jM5kXPp4h8CgBI_olJAB&start=110&sa=N&ved=0ahUKEwj67p-Fk7rgAhUHIIgKHQ80BRI4ZBDw0wMIXA
current page is 12
email found fullhousedenim_workshop@yahoo.com.hk
save to mongodb
no email found
email found denimwerk@yahoo.comf
save to mongodb
email found gjcpakenham@yahoo.com
save to mongodb
email found mass.jeans@yahoo.com
save to mongodb
email found europajeans@yahoo.com
save to mongodb
email found rightplantplace@yahoo.com
save to mongodb
email found fullhousedenim_workshop@yahoo.com.hk
save to mongodb
email found babu22@yahoo.comO
save to mongodb
no email found
next page url: https://www.google.com.hk/search?q=Jeans+site:facebook.com+%22%40yahoo.com%22&safe=strict&ei=vc5kXJH3NYP8wQPj47IY&start=120&sa=N&ved=0ahUKEwjRyYOdk7rgAhUDfnAKHeOxDAM4bhDw0wMIaA
open crawl page... https://www.google.com.hk/search?q=Jeans+site:facebook.com+%22%40yahoo.com%22&safe=strict&ei=vc5kXJH3NYP8wQPj47IY&st

open crawl page... https://www.google.com.hk/search?q=Jeans+site:facebook.com+%22%40yahoo.com%22&safe=strict&ei=3dBkXPewDcLX-Qb1xomQDg&start=220&sa=N&ved=0ahUKEwi3k46glbrgAhXCa94KHXVjAuI40gEQ8NMDCF4
current page is 23
email found fullhousedenim_workshop@yahoo.com.hk
save to mongodb
no email found
no email found
email found latifjeans@yahoo.com.br
save to mongodb
no email found
email found fullhousedenim_workshop@yahoo.com.hk
save to mongodb
email found moyaarts@yahoo.como
save to mongodb
email found nnikki25@yahoo.comf
save to mongodb
no email found
no email found
next page url: https://www.google.com.hk/search?q=Jeans+site:facebook.com+%22%40yahoo.com%22&safe=strict&ei=D9FkXNHXCY6moATvwJqwAg&start=230&sa=N&ved=0ahUKEwiRm_a3lbrgAhUOE4gKHW-gBiY43AEQ8NMDCGI
open crawl page... https://www.google.com.hk/search?q=Jeans+site:facebook.com+%22%40yahoo.com%22&safe=strict&ei=D9FkXNHXCY6moATvwJqwAg&start=230&sa=N&ved=0ahUKEwiRm_a3lbrgAhUOE4gKHW-gBiY43AEQ8NMDCGI
current page is 24
no email found
n

open crawl page... https://www.google.com.hk/search?q=Jeans+site:Instagram.com+%22%40yahoo.com%22&safe=strict&ei=CNNkXMDXEIXmwQPro5PABw&start=50&sa=N&ved=0ahUKEwjA--Ool7rgAhUFc3AKHevRBHg4KBDw0wMIcQ
current page is 6
email found chariotjeansco.info@yahoo.come
save to mongodb
email found themottas@yahoo.com.br
save to mongodb
email found hcutohclothing@yahoo.comD
save to mongodb
no email found
email found nakedfarmer@yahoo.comT
save to mongodb
email found manguraidenim@yahoo.com
save to mongodb
no email found
email found paul.fuentebella@yahoo.comI
save to mongodb
email found delightfultreatsbycmg@yahoo.com
save to mongodb
email found Isro.hadi@yahoo.comk
save to mongodb
next page url: https://www.google.com.hk/search?q=Jeans+site:Instagram.com+%22%40yahoo.com%22&safe=strict&ei=MtNkXOXWBdvAoAS2rJugCA&start=60&sa=N&ved=0ahUKEwjlt9y8l7rgAhVbIIgKHTbWBoQ4MhDw0wMIcQ
open crawl page... https://www.google.com.hk/search?q=Jeans+site:Instagram.com+%22%40yahoo.com%22&safe=strict&ei=MtNkXOXWBdvAoAS

current page is 15
email found fashiongz007@yahoo.com
save to mongodb
email found artalidia@yahoo.comwww.tinkerlust.com
save to mongodb
email found vivianudoh53@yahoo.comwww.youtube.com
save to mongodb
email found theresa_ascani@yahoo.com
save to mongodb
email found brookej423@yahoo.com
save to mongodb
email found Diegobotero71@yahoo.comi
save to mongodb
email found rueljeans@yahoo.coml
save to mongodb
email found LCwilsonlifecoach@yahoo.com-LC
save to mongodb
email found philheh@yahoo.com
save to mongodb
email found hellonewstore@yahoo.com
save to mongodb
next page url: https://www.google.com.hk/search?q=Jeans+site:Instagram.com+%22%40yahoo.com%22&safe=strict&ei=K9VkXLrrKMzn-AbsyI_4BQ&start=150&sa=N&ved=0ahUKEwj6rOatmbrgAhXMM94KHWzkA184jAEQ8NMDCHA
open crawl page... https://www.google.com.hk/search?q=Jeans+site:Instagram.com+%22%40yahoo.com%22&safe=strict&ei=K9VkXLrrKMzn-AbsyI_4BQ&start=150&sa=N&ved=0ahUKEwj6rOatmbrgAhXMM94KHWzkA184jAEQ8NMDCHA
current page is 16
email found angelique.

open crawl page... https://www.google.com.hk/search?q=Jeans+site:twitter.com+%22%40yahoo.com%22&safe=strict&ei=59ZkXOuKB4Hm-Aao6pmICg&start=50&sa=N&ved=0ahUKEwirmqCBm7rgAhUBM94KHSh1BqE4KBDw0wMIhgE
current page is 6
email found elefvog@yahoo.com
save to mongodb
email found design_erjeans@yahoo.com
save to mongodb
email found k.banks100k@yahoo.com
save to mongodb
email found Putrapoo@yahoo.com
save to mongodb
email found Putrapoo@yahoo.comh
save to mongodb
email found Putrapoo@yahoo.com
save to mongodb
email found Mark23320@yahoo.com
save to mongodb
email found inazallya@yahoo.com
save to mongodb
no email found
email found Ygaddiebookings@yahoo.com
save to mongodb
next page url: https://www.google.com.hk/search?q=Jeans+site:twitter.com+%22%40yahoo.com%22&safe=strict&ei=GNdkXJPzAs-h-Qau5rmQDQ&start=60&sa=N&ved=0ahUKEwiT38qYm7rgAhXPUN4KHS5zDtI4MhDw0wMIeg
open crawl page... https://www.google.com.hk/search?q=Jeans+site:twitter.com+%22%40yahoo.com%22&safe=strict&ei=GNdkXJPzAs-h-Qau5rmQDQ&sta

current page is 16
no email found
no email found
email found japhers@yahoo.com
save to mongodb
email found SamirFortune12@Yahoo.comF
save to mongodb
email found Monroestevens306@yahoo.com
save to mongodb
email found Lacaponebusiness@yahoo.comI
save to mongodb
email found katarina_williams5@yahoo.com
save to mongodb
email found nuel_p@yahoo.com
save to mongodb
email found Mhiris@yahoo.com
save to mongodb
email found snjeansle@yahoo.com
save to mongodb
next page url: https://www.google.com.hk/search?q=Jeans+site:twitter.com+%22%40yahoo.com%22&safe=strict&ei=KtlkXKGbLI2i-QaizKK4Dg&start=160&sa=N&ved=0ahUKEwih2NCVnbrgAhUNUd4KHSKmCOc4lgEQ8NMDCIgB
open crawl page... https://www.google.com.hk/search?q=Jeans+site:twitter.com+%22%40yahoo.com%22&safe=strict&ei=KtlkXKGbLI2i-QaizKK4Dg&start=160&sa=N&ved=0ahUKEwih2NCVnbrgAhUNUd4KHSKmCOc4lgEQ8NMDCIgB
current page is 17
email found snjeansle@yahoo.com
save to mongodb
email found ShelbyIveyChristie@yahoo.com
save to mongodb
email found Blokmanagement@

open crawl page... https://www.google.com.hk/search?q=Jeans+site:tumblr.com+%22%40yahoo.com%22&safe=strict&ei=a9tkXJKREdHs-Qat7ZHQAw&start=80&sa=N&ved=0ahUKEwjS8saon7rgAhVRdt4KHa12BDo4RhDw0wMIkwE
current page is 9
email found hermac1979@yahoo.com
save to mongodb
email found yoshidah@jeans.ocn.ne.jp
save to mongodb
no email found
email found Ghostletics@yahoo.com
save to mongodb
email found rwalkeryn@yahoo.como
save to mongodb
email found rawdenimhouse@yahoo.comi
save to mongodb
email found rawdenimhouse@yahoo.comi
save to mongodb
email found rawdenimhouse@yahoo.comi
save to mongodb
email found askgqcreek@yahoo.com
save to mongodb
email found rawdenimhouse@yahoo.comi
save to mongodb
next page url: https://www.google.com.hk/search?q=Jeans+site:tumblr.com+%22%40yahoo.com%22&safe=strict&ei=p9tkXJ6VFczU-QaL47g4&start=90&sa=N&ved=0ahUKEwjehJnFn7rgAhVMat4KHYsxDgc4UBDw0wMIkQE
open crawl page... https://www.google.com.hk/search?q=Jeans+site:tumblr.com+%22%40yahoo.com%22&safe=strict&ei=p9tkXJ6VF

open crawl page... https://www.google.com.hk/search?q=Jeans+site:linkedin.com+%22%40yahoo.com%22&safe=strict&ei=Xd1kXL2oAork-Aa18omgAg&start=80&sa=N&ved=0ahUKEwj9yvOVobrgAhUKMt4KHTV5AiQ4RhDw0wMIcA
current page is 9
email found mehedihasan_111@yahoo.comM
save to mongodb
email found praman_19@yahoo.com
save to mongodb
email found info@ayanenterprise.com
save to mongodb
email found hyunsik84@yahoo.com
save to mongodb
email found ksokolovsky@yahoo.com
save to mongodb
email found scottmdarlington@yahoo.com
save to mongodb
email found r4utraders@yahoo.como
save to mongodb
email found melissa_turner1@yahoo.com
save to mongodb
email found frecklebandit@yahoo.com
save to mongodb
email found dwipstar@yahoo.com
save to mongodb
next page url: https://www.google.com.hk/search?q=Jeans+site:linkedin.com+%22%40yahoo.com%22&safe=strict&ei=k91kXPfqMJbr-Qa455rIDQ&start=90&sa=N&ved=0ahUKEwi3gIKwobrgAhWWdd4KHbizBtk4UBDw0wMIdg
open crawl page... https://www.google.com.hk/search?q=Jeans+site:linkedin.com+%22

current page is 18
no email found
no email found
email found CindyJACarlson@yahoo.com
save to mongodb
email found uzairuddin64@yahoo.com
save to mongodb
email found sarahaghighi1994@Yahoo.com
save to mongodb
email found muday_bhaskar@yahoo.com
save to mongodb
email found necroproduct@yahoo.com
save to mongodb
there is no more pages.
getting email done


## google 关键字

In [26]:
import re


keyword_list = ['jeans site:amazon.com "@gmail.com"','jeans site:wish.com "@gmail.com"','jeans site:ebay.com "@gmail.com"','jeans site:facebook.com "@gmail.com"','jeans site:Instagram.com "@gmail.com"','jeans site:twitter.com "@gmail.com"','jeans site:tumblr.com "@gmail.com"','jeans site:linkedin.com "@gmail.com"','Jeans site:amazon.com @hotmail.com','Jeans site:wish.com @hotmail.com','Jeans site:ebay.com @hotmail.com','Jeans site:facebook.com @hotmail.com','Jeans site:Instagram.com @hotmail.com','Jeans site:twitter.com @hotmail.com','Jeans site:tumblr.com @hotmail.com','Jeans site:linkedin.com @hotmail.com','Jeans site:amazon.com @yahoo.com','Jeans site:wish.com @yahoo.com','Jeans site:ebay.com @yahoo.com','Jeans site:facebook.com @yahoo.com','Jeans site:Instagram.com @yahoo.com','Jeans site:twitter.com @yahoo.com','Jeans site:tumblr.com @yahoo.com','Jeans site:linkedin.com @yahoo.com']
print(len(keyword_list))
for i in keyword_list:
    keyword = re.sub(r'site:|@|\"','',i,count=0)
    keywords = keyword.split()
    url_format = 'https://www.google.com.hk/search?&q={0[0]}+site%3A{0[1]}+%22%40{0[2]}%22&oq'.format((keywords))
    print(url_format)

24
https://www.google.com.hk/search?&q=jeans+site%3Aamazon.com+%22%40gmail.com%22&oq
https://www.google.com.hk/search?&q=jeans+site%3Awish.com+%22%40gmail.com%22&oq
https://www.google.com.hk/search?&q=jeans+site%3Aebay.com+%22%40gmail.com%22&oq
https://www.google.com.hk/search?&q=jeans+site%3Afacebook.com+%22%40gmail.com%22&oq
https://www.google.com.hk/search?&q=jeans+site%3AInstagram.com+%22%40gmail.com%22&oq
https://www.google.com.hk/search?&q=jeans+site%3Atwitter.com+%22%40gmail.com%22&oq
https://www.google.com.hk/search?&q=jeans+site%3Atumblr.com+%22%40gmail.com%22&oq
https://www.google.com.hk/search?&q=jeans+site%3Alinkedin.com+%22%40gmail.com%22&oq
https://www.google.com.hk/search?&q=Jeans+site%3Aamazon.com+%22%40hotmail.com%22&oq
https://www.google.com.hk/search?&q=Jeans+site%3Awish.com+%22%40hotmail.com%22&oq
https://www.google.com.hk/search?&q=Jeans+site%3Aebay.com+%22%40hotmail.com%22&oq
https://www.google.com.hk/search?&q=Jeans+site%3Afacebook.com+%22%40hotmail.com%22&oq
htt

## google人机身份验证 (未解决)

In [9]:
from selenium import webdriver
import time,random,re,csv,pymongo
from pyquery import PyQuery as pq
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

url = 'https://www.inipec.gov.it/cerca-pec/-/pecs/companies'
driver = webdriver.Chrome()
driver.get(url)
time.sleep(5)
driver.switch_to.frame(driver.find_elements_by_tag_name("iframe")[0])
time.sleep(2)
driver.find_element_by_css_selector('div.recaptcha-checkbox-checkmark').click()

## 数据库 去重，调用 update()
首先根据昵称和正文来查询信息，如果信息不存在，则插入数据，否则更新数据。这个操作地关键点是第三个参数 True, 此参数设置为 True, 这可以实现存在即更新，不存在则插入的操作。

In [ ]:
self.collection.update_one({'nickname':nickname,'content':content},{'$set'；data},True)

## mongodb 数据导出 csv

导出csv格式的命令：

mongoexport -d 数据库名字 -c 集合的名字 --type=csv -f 导出的列名 -o 导出文件名

mongoexport -d shopcrawl -c email --type=csv -f email -o gmail_test.csv


In [5]:
import csv,re

with open(r'gmails_jeans.csv','a',encoding='utf-8') as w:
    with open(r'gmail_jeans.csv','r',encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
                print(row[1])
                try:
                    email = re.search(r'[A-Za-z0-9_.]+@[A-Za-z0-9]+\.com',row[1]).group(0)
                    print(email)
                    w.write('{},{}\n'.format('jeans',email))
                except:
                    pass

lbts@gmail.com
lbts@gmail.com
Jsokolis@gmail.com
Jsokolis@gmail.com
mikimarcu256@gmail.com
mikimarcu256@gmail.com
jasgoodonline@gmail.com
jasgoodonline@gmail.com
magekagura9@gmail.com
magekagura9@gmail.com
breakthewineglass@gmail.comi
breakthewineglass@gmail.com
sarifhidayat808@gmail.com
sarifhidayat808@gmail.com
toqeer65@gmail.com
toqeer65@gmail.com
staffkidsshoppenj@gmail.coms
staffkidsshoppenj@gmail.com
evertvear@gmail.com
evertvear@gmail.com
tnvo18@gmail.comI
tnvo18@gmail.com
chamsimo17@gmail.com
chamsimo17@gmail.com
cocoscollection88@gmail.comT
cocoscollection88@gmail.com
dakilivewallpapers@gmail.com
dakilivewallpapers@gmail.com
vasishtanagalla@gmail.com
vasishtanagalla@gmail.com
breakthewineglass@gmail.comi
breakthewineglass@gmail.com
Giftmantrausa@gmail.com
Giftmantrausa@gmail.com
jeannineyoung1@gmail.com
jeannineyoung1@gmail.com
munwarapps@gmail.com
munwarapps@gmail.com
doodlesanddots2@gmail.comw
doodlesanddots2@gmail.com
jeannineyoung1@gmail.com
jeannineyoung1@gmail.com
vypera

rsum850@gmail.com
contact.nobrandedon@gmail.com
contact.nobrandedon@gmail.com
soundcrue.sapporo.jp@gmail.com
soundcrue.sapporo.jp@gmail.com
throatrecords@gmail.com
throatrecords@gmail.com
lisatrimm.ssi@gmail.com
lisatrimm.ssi@gmail.com
meximcom@gmail.com
meximcom@gmail.com
craiserjeans@gmail.com
craiserjeans@gmail.com
chrisbellaijk@gmail.com
chrisbellaijk@gmail.com
Deedee7899@gmail.com
Deedee7899@gmail.com
wiremeshweld@163.com
wiremeshweld@163.com
apparel1822@gmail.comR
apparel1822@gmail.com
info.thetherapyplace@gmail.como
info.thetherapyplace@gmail.com
hlee.denim@gmail.com
hlee.denim@gmail.com
marksapparel@gmail.comM
marksapparel@gmail.com
jyotsna.amritraj@gmail.com
jyotsna.amritraj@gmail.com
usajeanswear1@gmail.com
usajeanswear1@gmail.com
newartmannequins@gmail.com
newartmannequins@gmail.com
business.partners.jeans@gmail.com
business.partners.jeans@gmail.com
scott.morrison@3x1.us
vazquezruizjairo@gmail.com
vazquezruizjairo@gmail.com
jyotsna.amritraj@gmail.com
jyotsna.amritraj@gmail.c

Kteplitsky@hotmail.com
Mecanico.ventas@hotmail.com
Mecanico.ventas@hotmail.com
vsactu@hotmail.com
vsactu@hotmail.com
wizardembroidery@hotmail.com
wizardembroidery@hotmail.com
floralsandcorals@hotmail.comI
floralsandcorals@hotmail.com
varisj43@hotmail.comY
varisj43@hotmail.com
evarozee@hotmail.com-Plea
evarozee@hotmail.com
josiel.jeans@hotmail.com
josiel.jeans@hotmail.com
live.in.the.moment.tokyo@hotmail.compic.twitter.com
live.in.the.moment.tokyo@hotmail.com
cgonzalezr22@hotmail.com
cgonzalezr22@hotmail.com
Jamesholtx@hotmail.com
Jamesholtx@hotmail.com
kataotalvaro@hotmail.com
kataotalvaro@hotmail.com
editorial_tike@hotmail.com
editorial_tike@hotmail.com
miscelanea.bauru@hotmail.comw
miscelanea.bauru@hotmail.com
atendimentoonlinedijulys@hotmail.com
atendimentoonlinedijulys@hotmail.com
vsactu@hotmail.com
vsactu@hotmail.com
pedidosangeles@hotmail.comy
pedidosangeles@hotmail.com
thebrightongirl@hotmail.com
thebrightongirl@hotmail.com
LaurenVickersModel@hotmail.com
LaurenVickersModel@hotma

fashiongz007@yahoo.com
artalidia@yahoo.comwww.tinkerlust.com
artalidia@yahoo.com
vivianudoh53@yahoo.comwww.youtube.com
vivianudoh53@yahoo.com
theresa_ascani@yahoo.com
theresa_ascani@yahoo.com
brookej423@yahoo.com
brookej423@yahoo.com
Diegobotero71@yahoo.comi
Diegobotero71@yahoo.com
rueljeans@yahoo.coml
rueljeans@yahoo.com
LCwilsonlifecoach@yahoo.com-LC
LCwilsonlifecoach@yahoo.com
philheh@yahoo.com
philheh@yahoo.com
hellonewstore@yahoo.com
hellonewstore@yahoo.com
angelique.manto@yahoo.com
angelique.manto@yahoo.com
femmefatalestyle@yahoo.com
femmefatalestyle@yahoo.com
lindseylesliestuart@yahoo.com
lindseylesliestuart@yahoo.com
gentryjudd@yahoo.com
gentryjudd@yahoo.com
bigfattysck@yahoo.com
bigfattysck@yahoo.com
Email-capofashion@yahoo.com
capofashion@yahoo.com
HotRodDirty@yahoo.com
HotRodDirty@yahoo.com
Salkhawaja@yahoo.com
Salkhawaja@yahoo.com
selindavo@yahoo.comwww.tinkerlust.com
selindavo@yahoo.com
KC.johns@yahoo.com
KC.johns@yahoo.com
malkisigal@yahoo.com
malkisigal@yahoo.com
infobel